In [ ]:
minimum_cut

In [1]:
import torch
import torch.nn.functional as F
from dgl.nn import GraphConv
import dgl
import torch.nn as nn
from itertools import chain, islice, combinations

class GCN_dev(torch.nn.Module):
    def __init__(self, in_feats, hidden_size, number_classes, dropout, device):
        super(GCN_dev, self).__init__()
        self.dropout_frac = dropout
        self.conv1 = GraphConv(in_feats, hidden_size).to(device)
        self.conv2 = GraphConv(hidden_size, number_classes).to(device)

    def forward(self, g, inputs):
        h = self.conv1(g, inputs)
        h = torch.relu(h)
        h = F.dropout(h, p=self.dropout_frac, training=self.training)
        h = self.conv2(g, h)
        h = torch.tanh(h)
        return h

# Manual run of the Hamiltonian loss function on the graph we created earlier

# Create a simple DGL graph
edges = [(0, 1), (0, 2), (0, 3), (2, 3)]  # Edges as per the earlier example graph
g = dgl.graph(edges)

# Edge weight matrix as per the earlier example graph
weights = torch.tensor([
    [0, 1, 3, 2],
    [1, 0, 0, 0],
    [3, 0, 0, 1],
    [2, 0, 1, 0]
], dtype=torch.float32)

# Add self-edges to include self-connections in the GCN
g = dgl.add_self_loop(g)

# Assuming a device (for simplicity, using CPU here)
device = torch.device('cpu')

# Instantiate the GCN model
gcn_model = GCN_dev(in_feats=16, hidden_size=2, number_classes=1, dropout=0.5, device=device)

#gcn_model = gcn_model.type(torch_dtype).to(torch_device)
embed = nn.Embedding(4, 16)
#embed = embed.type(torch_dtype).to(torch_device)

# set up Adam optimizer
params = chain(gcn_model.parameters(), embed.parameters())

opt_params = {'lr': 0.01}
optimizer = torch.optim.Adam(params, **opt_params)

# Example input features (randomly initialized for this demonstration)
features = torch.rand((4, 16), device=device)  # 5 nodes with 25 features each

# Forward pass through the GCN
# outputs = gcn_model(g, features)

# Hamiltonian loss calculation
def hamiltonian_loss(h, W):
    # Compute the edge-cut term HB
    probs_diff = torch.abs(h.unsqueeze(-1) - h.unsqueeze(0))
    # Multiply by the weights matrix W
    HB = torch.sum(W * probs_diff) / 2  # Divide by 2 to account for double-counting edges
    return HB

def hamiltonian_loss5(h, W):
    # Compute the edge-cut term HB
    probs_diff = torch.abs(h.unsqueeze(-1) - h.unsqueeze(0))
    # Multiply by the weights matrix W
    HB = torch.sum(W * probs_diff) / 2  # Divide by 2 to account for double-counting edges

    terminal_loss = torch.abs(h[0] - h[3])
    HB += (1 * (1 - terminal_loss))[0]

    return HB

def hamiltonian_loss1(h, W, A=1, B=1):
    """
    Compute the Hamiltonian loss for a given set of node probabilities and edge weights.
    This function is designed to be differentiable to retain gradients for backpropagation.

    Parameters:
    h (Tensor): The output probabilities from the GCN, should require gradient.
    W (Tensor): The weight matrix for the edges, should not require gradient.
    A (float): The weighting factor for the HA term.
    B (float): The weighting factor for the HB term.

    Returns:
    Tensor: The computed Hamiltonian loss.
    """
    # Convert the probabilities to 'spins'. Values > 0.5 are converted to 1, else -1.
    spins = torch.where(h > 0.5, 1, -1).float()

    # HA is a penalty term for the imbalance of vertices in two partitions
    balance = torch.sum(spins, dim=0)  # Sum spins for balance
    HA = balance ** 2 if balance.numel() % 2 == 0 else balance ** 2 / 2

    # HB is the edge-cut term, calculated using the absolute difference of 'spins'
    spins_diff = torch.abs(spins.unsqueeze(-1) - spins.unsqueeze(0))
    HB = torch.sum(W * spins_diff) / 2  # Divide by 2 to account for double-counting edges

    # The Hamiltonian loss is a combination of HA and HB, weighted by A and B
    H = A * HA + B * HB
    return H


def hamiltonian_loss2(h, W, A=1, B=1):
    """
    Compute the Hamiltonian loss for a given set of node probabilities and edge weights.
    This function is designed to be differentiable to retain gradients for backpropagation.


    Parameters:
    h (Tensor): The output probabilities from the GCN, should require gradient.
    W (Tensor): The weight matrix for the edges, should not require gradient.
    A (float): The weighting factor for the HA term.
    B (float): The weighting factor for the HB term.

    Returns:
    Tensor: The computed Hamiltonian loss.
    """
    num_nodes = h.size(0)
    ideal_balance = num_nodes / 2.0

    # HA is a penalty term for the deviation of sum of probabilities from the ideal balance
    HA = (torch.sum(h) - ideal_balance) ** 2

    # HB is the edge-cut term, calculated using the absolute difference of probabilities
    probs_diff = torch.abs(h.unsqueeze(-1) - h.unsqueeze(0))
    HB = torch.sum(W * probs_diff) / 2  # Divide by 2 to account for double-counting edges

    # The Hamiltonian loss is a combination of HA and HB, weighted by A and B
    H = A * HA + B * HB
    return H
# # Calculate the loss
# loss = hamiltonian_loss(outputs, weights)
# #loss2 = hamiltonian_loss1(outputs, weights)
# optimizer.zero_grad()
# loss.backward()
# optimizer.step()
#
# outputs = gcn_model(g, features)
# loss = hamiltonian_loss(outputs, weights)
#loss2 = hamiltonian_loss1(outputs, weights)

for i in range(100):
    outputs = gcn_model(g, features)
    loss = hamiltonian_loss2(outputs, weights)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Display the outputs and loss
outputs, loss #, loss2


(tensor([[0.4946],
         [0.4946],
         [0.4946],
         [0.4946]], grad_fn=<TanhBackward0>),
 tensor(0.0005, grad_fn=<AddBackward0>))

In [47]:
import torch
import torch.nn.functional as F
from dgl.nn import GraphConv
import dgl
import torch.nn as nn

class GCN_dev(torch.nn.Module):
    def __init__(self, in_feats, hidden_size, number_classes, dropout, device):
        super(GCN_dev, self).__init__()
        self.dropout_frac = dropout
        self.conv1 = GraphConv(in_feats, hidden_size).to(device)
        self.conv2 = GraphConv(hidden_size, number_classes).to(device)
        self.skip_connection = nn.Linear(in_feats, number_classes).to(device)

    def forward(self, g, inputs):
        x = inputs
        h = self.conv1(g, inputs)
        h = torch.relu(h)
        h = F.dropout(h, p=self.dropout_frac, training=self.training)
        h = self.conv2(g, h)

        # Skip connection from input to output
        skip = self.skip_connection(x)
        h = h + skip

        h = torch.tanh(h)
        return h

# Rest of your setup code remains the same...

# Learning rate scheduler setup
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

# Training loop with early stopping
best_loss = float('inf')
epochs_no_improve = 0
n_epochs_stop = 500  # Number of epochs to wait before stopping without improvement

for i in range(10000):
    outputs = gcn_model(g, features)
    loss = hamiltonian_loss5(outputs, weights)

    if loss < best_loss:
        best_loss = loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve == n_epochs_stop:
            print("Early stopping")
            break

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

# Display the outputs and loss
outputs, loss


Early stopping


(tensor([[0.5000],
         [0.5000],
         [0.5000],
         [0.5000]], grad_fn=<TanhBackward0>),
 tensor(1., grad_fn=<AddBackward0>))

In [48]:
import torch
import torch.nn.functional as F
from dgl.nn import GraphConv
import dgl
import torch.nn as nn
import numpy as np
import random
from collections import defaultdict

# Define the GCN model
class GCN_dev(nn.Module):
    def __init__(self, in_feats, hidden_size, number_classes, dropout, device):
        super(GCN_dev, self).__init__()
        self.dropout_frac = dropout
        self.conv1 = GraphConv(in_feats, hidden_size).to(device)
        self.conv2 = GraphConv(hidden_size, number_classes).to(device)

    def forward(self, g, inputs):
        h = self.conv1(g, inputs)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout_frac, training=self.training)
        h = self.conv2(g, h)
        h = torch.sigmoid(h)
        return h

# Graph environment for RL
class GraphEnv:
    def __init__(self, g, weights):
        self.g = g
        self.weights = weights
        self.state = torch.rand((g.num_nodes(), in_feats), device=device)
        self.num_nodes = g.num_nodes()

    def reset(self):
        self.state = torch.rand((self.num_nodes, in_feats), device=device)
        return self.state

    def step(self, action):
        # Action: flip the partition of a node
        self.state[action] = 1 - self.state[action]
        new_cut_size = self.calculate_cut_size()
        reward = -new_cut_size
        return self.state, reward

    def calculate_cut_size(self):
        # Calculate the cut size based on current state
        cut_size = 0
        src, dst = self.g.edges()
        for u, v in zip(src.tolist(), dst.tolist()):
            if self.state[u] != self.state[v]:
                cut_size += self.weights[u][v].item()
        return cut_size

# Create a simple graph
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]  # Define the edges of the graph
g = dgl.graph(edges)  # Create a DGL graph
g = dgl.add_self_loop(g)  # Add self-loops

# Edge weight matrix
weights = torch.tensor([
    [0, 1, 0, 1],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [1, 0, 1, 0]
], dtype=torch.float32)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate the GCN model
in_feats = 5  # Number of input features for each node
hidden_size = 2  # Size of the hidden layer
number_classes = 1  # Number of output classes (probabilities)
dropout = 0.5  # Dropout rate

gcn_model = GCN_dev(in_feats, hidden_size, number_classes, dropout, device).to(device)

# Q-Learning Parameters
learning_rate = 0.01
discount_factor = 0.99
exploration_rate = 0.1

# Q-Table for storing Q-Values
Q_table = defaultdict(float)

# Instantiate the environment
env = GraphEnv(g, weights)

# Reinforcement Learning Training Loop
for epoch in range(1000):
    current_state = env.reset()
    done = False

    while not done:
        # Choose an action based on the current state
        if random.uniform(0, 1) < exploration_rate:
            action = random.choice(range(g.num_nodes()))  # Explore
        else:
            # Exploit based on Q-Values
            action = np.argmax([Q_table[(tuple(current_state), a)] for a in range(g.num_nodes())])

        # Take the action and observe the new state and reward
        new_state, reward = env.step(action)

        # Update the Q-Table
        old_value = Q_table[(tuple(current_state), action)]
        next_max = max([Q_table[(tuple(new_state), a)] for a in range(g.num_nodes())])
        Q_table[(tuple(current_state), action)] = old_value + learning_rate * (reward + discount_factor * next_max - old_value)

        current_state = new_state

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Current Cut Size: {env.calculate_cut_size()}")



RuntimeError: Boolean value of Tensor with more than one value is ambiguous